# Stock Price Prediction using LSTM
# 
## This notebook demonstrates how to predict stock prices using a Long Short-Term Memory (LSTM) neural network.

### Importing Required Libraries

In [ ]:
!pip install -U -r requirements.txt

In [ ]:
# Numerical computing
import numpy as np

# Data manipulation and analysis
import pandas as pd

# Data visualization
import matplotlib.pyplot as plt

# Date and time operations
import datetime as dt

# Machine learning and deep learning
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer

# Stock data retrieval
import yfinance as yf

### Loading and Preparing Data

In [ ]:
#Stock to predict
COMPANY = 'CBA.AX'
#Train start date
TRAIN_START = '2020-01-01'
#Train end date
TRAIN_END = '2023-08-01'
#Price value to predict
PRICE_VALUE = "Close"
#Number of days to predict
PREDICTION_DAYS = 60

# Download stock data
data = yf.download(COMPANY, TRAIN_START, TRAIN_END)

# Prepare data for scaling using `MinMaxScaler()`
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[PRICE_VALUE].values.reshape(-1, 1))

# Prepare training data
x_train = []
y_train = []

scaled_data = scaled_data[:,0]
for x in range(PREDICTION_DAYS, len(scaled_data)):
    x_train.append(scaled_data[x-PREDICTION_DAYS:x])
    y_train.append(scaled_data[x])

# Convert to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

### Building and Training the Model

In [ ]:
# Using Sequential model
model = Sequential()

# Adding LSTM layers
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
# For each LSTM layer, we add a dropout layer
model.add(Dropout(0.2))
# Adding LSTM layers
model.add(LSTM(units=50, return_sequences=True))
# For each LSTM layer, we add a dropout layer
model.add(Dropout(0.2))
# Adding LSTM layers
model.add(LSTM(units=50))
# For each LSTM layer, we add a dropout layer
model.add(Dropout(0.2))
# Adding Dense layer
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, epochs=25, batch_size=32)

### Testing the Model

In [ ]:
# Test start date
TEST_START = '2023-08-02'
# Test end date
TEST_END = '2024-07-02'

# Download test data
test_data = yf.download(COMPANY, TEST_START, TEST_END)
# Actual prices
actual_prices = test_data[PRICE_VALUE].values

# Concatenate training and test data
total_dataset = pd.concat((data[PRICE_VALUE], test_data[PRICE_VALUE]), axis=0)

# Prepare test data
model_inputs = total_dataset[len(total_dataset) - len(test_data) - PREDICTION_DAYS:].values
# Reshape the data
model_inputs = model_inputs.reshape(-1, 1)
# Scale the data
model_inputs = scaler.transform(model_inputs)

# Make predictions
x_test = []
for x in range(PREDICTION_DAYS, len(model_inputs)):
    # Append the last PREDICTION_DAYS days to x_test
    x_test.append(model_inputs[x - PREDICTION_DAYS:x, 0])

# Convert to numpy arrays
x_test = np.array(x_test)
# Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Predict the prices
predicted_prices = model.predict(x_test)
# Inverse transform the data
predicted_prices = scaler.inverse_transform(predicted_prices)

### Visualizing the Results

In [ ]:
# Plot the actual prices
plt.plot(actual_prices, color="black", label=f"Actual {COMPANY} Price")
# Plot the predicted prices
plt.plot(predicted_prices, color="green", label=f"Predicted {COMPANY} Price")
# Set the title
plt.title(f"{COMPANY} Share Price")
# Set the x-axis label
plt.xlabel("Time")
# Set the y-axis label
plt.ylabel(f"{COMPANY} Share Price")
# Add a legend
plt.legend()
# Show the plot
plt.show()
### Predicting the Next Day's Price
plt.show()

### Predicting the Next Day's Price

In [ ]:
# Get the last PREDICTION_DAYS days
real_data = [model_inputs[len(model_inputs) - PREDICTION_DAYS:, 0]]
# Convert to numpy array
real_data = np.array(real_data)
# Reshape the data
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

# Predict the next day's price
prediction = model.predict(real_data)
# Inverse transform the data
prediction = scaler.inverse_transform(prediction)
# Print the prediction
print(f"Prediction: {prediction}")